In [ ]:
! pip install sshtunnel
! pip install pymongo

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

from sshtunnel import SSHTunnelForwarder
import pymongo
from pymongo.operations import UpdateMany, InsertOne

### Classe de connection à la base MongoDB

In [2]:
class Connection_MongoDB:

    def __init__(self, bridge_host="137.194.211.146", bridge_username="ubuntu", remote_server_name="tp-hadoop-51", remote_server_port=27017):
        self.server = SSHTunnelForwarder(
            bridge_host,#"137.194.211.146",
            ssh_username=bridge_username,#"ubuntu",
            remote_bind_address=(remote_server_name,remote_server_port)#("tp-hadoop-51", 27017),
            #local_bind_address=('127.0.0.1', 10022)
        ) 
        self.server.start()
        self.session = pymongo.MongoClient(self.server.local_bind_hosts[0], self.server.local_bind_port)
    
    def __del__(self):
        self.server.stop()
        
    def get_collection(self, db_name = "test", collection_name="events"):
        db = self.session[db_name]
        self.collection = db[collection_name]
        return self
    
    def find(self, request):
        return self.collection.find(request)
    
    def aggregate(self, request):
        return self.collection.aggregate(request, allowDiskUse=True)
    
    def insert_many(self, request):
        return self.collection.insert_many(request)
    
    def remove_all(self):
        return self.collection.delete_many( { } )
    
    def create_index(self, fields, unique=True):
        return self.collection.create_index(fields, unique=unique)
    
    def update_many(self, filter_request, update_request):
        return self.collection.update_many(filter_request, update_request)
    
    def bulk_write(self, requests, ordered=False):
        return self.collection.bulk_write(requests, ordered=ordered)
    
    
mention_collection = Connection_MongoDB().get_collection(db_name = "final", collection_name="mention2")

### Chargement des données dans la base

In [ ]:
# création des index pour le chargement
mention_collection.create_index([("MentionIdentifier", 1) ], unique=False )
mention_collection.create_index([("GLOBALEVENTID", 1) ], unique=False )

In [ ]:
df = pd.read_csv("http://data.gdeltproject.org/gdeltv2/masterfilelist-translation.txt",
                 delimiter = " ",
                 header= None,
                 names = ["ID1","ID2","URL"]
                )

#### Ajout des mentions

In [ ]:
%%time

# extraction de toutes les url des mentions de janvier à mars 2022
df_mention_list = df[df.URL.str.contains('2022[0-9]{10}.*mentions.CSV',regex= True, na=False)]

#pour chaque fichier de mention on charge le document CSV
for i, mention_url in enumerate(df_mention_list.URL):
    try:
        if i % 500 == 0:
            print(mention_url)
        #    break

        # Chargements des données des mentions
        mention_colnames = ["GLOBALEVENTID","EventTimeDate","MentionTimeDate","MentionType","MentionSourceName","MentionIdentifier","SentenceID","Actor1CharOffset","Actor2CharOffset","ActionCharOffset","InRawText","Confidence","MentionDocLen","MentionDocTone","MentionDocTranslationInfo","Extras"]
        mention_cols_to_keep = ['GLOBALEVENTID', 'MentionTimeDate', 'MentionIdentifier']
        df_mention_data = pd.read_csv(mention_url, sep="\t", names=mention_colnames, usecols=mention_cols_to_keep, header=None, encoding='latin')

        #Supprime les lignes dupliquées
        df_mention_data = df_mention_data.drop_duplicates()

        #on charge les données dans la base
        json_mention = df_mention_data.to_dict('records')
        #mention_collection.insert_many(json_mention)
        requests = []
        for m in json_mention:
            requests.append(InsertOne(m))
        mention_collection.bulk_write(requests, ordered=False)
    except:
        print("Probleme",mention_url)

#### Ajout des articles GKG (jointure par update)

In [ ]:
%%time

# extraction de toutes les url des articles gkg de 2022
df_gkg_list = df[df.URL.str.contains('2022[0-9]{10}.*gkg.csv',regex= True, na=False)]


#pour chaque fichier de mention on charge le document CSV
for i, gkg_url in enumerate(df_gkg_list.URL):
    
    try:
        if i % 500 == 0:
            print(gkg_url)

        # Chargements des données des mentions
        gkg_colnames = ["GKGRECORDID","DATE","SourceCollectionIdentifier","SourceCommonName","DocumentIdentifier","Counts","V2Counts","Themes","V2Themes","Locations","V2Locations","Persons","V2Persons","Organizations","V2Organizations","V2Tone","Dates","GCAM","SharingImage","RelatedImages","SocialImageEmbeds","SocialVideoEmbeds","Quotations","AllNames","Amounts","TranslationInfo","Extras"]
        gkg_cols_to_keep = ['GKGRECORDID', 'DATE', 'SourceCommonName', 'DocumentIdentifier', 'Themes', 'Persons', 'V2Tone', "TranslationInfo"]
        df_gkg_data = pd.read_csv(gkg_url, sep="\t", names=gkg_colnames, usecols=gkg_cols_to_keep, header=None, encoding='latin')
        #Supprime les lignes dupliquées
        df_gkg_data = df_gkg_data.drop_duplicates()

        #nettoyage des df
        df_gkg_data.TranslationInfo = df_gkg_data.TranslationInfo.str.extract(r'srclc:([a-z]{3}).*')
        df_gkg_data.Themes = df_gkg_data.Themes.str.split(";")
        df_gkg_data.Persons = df_gkg_data.Persons.str.split(";")
        df_gkg_data.V2Tone = df_gkg_data.V2Tone.str.split(",").str[0].astype(float)

        #on charge les données dans la base
        json_gkg = df_gkg_data.to_dict('records')
        requests = []
        for article in json_gkg:
            #mention_collection.update_many({"MentionIdentifier": article["DocumentIdentifier"]}, {"$set" : {"article":article}})
            requests.append(UpdateMany({"MentionIdentifier": article["DocumentIdentifier"]}, {"$set" : {"article":article}}))
        mention_collection.bulk_write(requests, ordered=False)
    except:
        print("probleme",gkg_url)

#### Ajout des événement (jointure par update)

In [ ]:
%%time

# extraction de toutes les url des evenement de 2022
df_event_list = df[df.URL.str.contains('2022[0-9]{10}.*export.CSV',regex= True, na=False)]

#pour chaque fichier de mention on charge le document CSV
for i, event_url in enumerate(df_event_list.URL):
    try:
        if i % 500 == 0:
            print(event_url)

        # Chargements des données des mentions
        event_colnames = "GLOBALEVENTID;SQLDATE;MonthYear;Year;FractionDate;Actor1Code;Actor1Name;Actor1CountryCode;Actor1KnownGroupCode;Actor1EthnicCode;Actor1Religion1Code;Actor1Religion2Code;Actor1Type1Code;Actor1Type2Code;Actor1Type3Code;Actor2Code;Actor2Name;Actor2CountryCode;Actor2KnownGroupCode;Actor2EthnicCode;Actor2Religion1Code;Actor2Religion2Code;Actor2Type1Code;Actor2Type2Code;Actor2Type3Code;IsRootEvent;EventCode;EventBaseCode;EventRootCode;QuadClass;GoldsteinScale;NumMentions;NumSources;NumArticles;AvgTone;Actor1Geo_Type;Actor1Geo_FullName;Actor1Geo_CountryCode;Actor1Geo_ADM1Code;Actor1Geo_ADM2Code;Actor1Geo_Lat;Actor1Geo_Long;Actor1Geo_FeatureID;Actor2Geo_Type;Actor2Geo_FullName;Actor2Geo_CountryCode;Actor2Geo_ADM1Code;Actor2Geo_ADM2Code;Actor2Geo_Lat;Actor2Geo_Long;Actor2Geo_FeatureID;ActionGeo_Type;ActionGeo_FullName;ActionGeo_CountryCode;ActionGeo_ADM1Code;ActionGeo_ADM2Code;ActionGeo_Lat;ActionGeo_Long;ActionGeo_FeatureID;DATEADDED;SOURCEURL".split(";")
        event_cols_to_keep = ['GLOBALEVENTID', 'SQLDATE', 'Actor1Geo_CountryCode', 'Actor2Geo_CountryCode', 'ActionGeo_CountryCode']
        df_event_data = pd.read_csv(event_url, sep="\t", names=event_colnames, usecols= event_cols_to_keep, header=None, encoding='latin')

        #Supprime les lignes dupliquées
        df_event_data = df_event_data.drop_duplicates()

        #nettoyage des df
        df_event_data.SQLDATE = df_event_data.SQLDATE.astype(str)

        #on charge les données dans la base
        json_event = df_event_data.to_dict('records')

        requests = []
        for event in json_event:
            #mention_collection.update_many({"GLOBALEVENTID": event["GLOBALEVENTID"]}, {"$set" : {"event":event}})
            requests.append(UpdateMany({"GLOBALEVENTID": event["GLOBALEVENTID"]}, {"$set" : {"event":event}}))
        mention_collection.bulk_write(requests, ordered=False)
    except:
        print("probleme",event_url)

### Requetage

**QUESTION 1:** afficher le nombre d’articles/évènements qu’il y a eu pour chaque triplet (jour, pays de l’évènement, langue de l’article).

In [ ]:
mention_collection.create_index([("event.SQLDATE", 1),("event.ActionGeo_CountryCode", 1),
                                 ("article.TranslationInfo", 1) ], unique=False )

In [ ]:
%%time

def get_nb_article_event(collection):
    result = collection.aggregate([
        { "$match" : { "event.SQLDATE" : {"$regex":"2022010[0-9]{1}"}} },
        { "$group" : { "_id" : { "date" : "$event.SQLDATE", 
                                "pays" : "$event.ActionGeo_CountryCode", 
                                "langue":"$article.TranslationInfo" }, 
                      "unique_article": {"$addToSet": "$MentionIdentifier"}, 
                      "unique_event": {"$addToSet": "$GLOBALEVENTID"} 
                     } 
        },
        { "$project" : { "date" : "$_id.date", 
                        "pays" : "$_id.pays", 
                        "langue":"$_id.langue", 
                        "nb_distinct_article" : {"$size":"$unique_article"}, 
                        "nb_distinct_event" : {"$size":"$unique_event"} 
                       } 
        }
    ])
    return pd.DataFrame(list(result))

get_nb_article_event(mention_collection)

**QUESTION 2:** pour un pays donné en paramètre, affichez les évènements qui y ont eu place triées par le nombre de mentions (tri décroissant); permettez une agrégation par jour/mois/année

In [ ]:
mention_collection.create_index([("event.ActionGeo_CountryCode", 1)], unique=False )

mention_collection.create_index([("event.GLOBALEVENTID", 1),("event.SQLDATE", 1) ], unique=False )


In [5]:
%%time

def find_events_from_country(collection, country="FR", time_granularity="day"):
    date_format="%Y-%m-%d"
    if time_granularity == "month":
        date_format="%Y-%m"
    if time_granularity == "year":
        date_format="%Y"
        
    result = collection.aggregate([
        { "$match" : { "event.ActionGeo_CountryCode" : country } },
        { "$group" : { "_id" : {"eventID": "$event.GLOBALEVENTID",
                                "date":{ "$dateToString": { "format": date_format, "date":{"$dateFromString":{
                                    "dateString": "$event.SQLDATE", "format": "%Y%m%d"}}}}
                               }, 
                      "count_mention":{ "$sum": 1} 
                     }
        }, 
        { "$project" : {"_id" : 0,
                        "eventID":"$_id.eventID", 
                        "date": "$_id.date",
                        "count_mention": { "$sum": "$count_mention"} 
                       }
        },
        {"$sort": {"count_mention": -1}}
    ])
    return pd.DataFrame(list(result))

df_events_from_country = find_events_from_country(mention_collection, country="FR", time_granularity="month")
df_events_from_country

CPU times: user 3.97 ms, sys: 2.38 ms, total: 6.35 ms
Wall time: 25.6 ms


""


In [ ]:
fig = px.bar(df_events_from_country, x="date", y="count_mention", color="count_mention", hover_name="eventID")
fig.show()

**QUESTION 3 :** pour une source de donnés passée en paramètre (gkg.SourceCommonName) affichez les thèmes, personnes, lieux dont les articles de cette sources parlent ainsi que le nombre d’articles et le ton moyen des articles (pour chaque thème/personne/lieu); permettez une agrégation par jour/mois/année.

In [ ]:
mention_collection.create_index([("article.SourceCommonName", 1)], unique=False )

In [ ]:
%%time

def article_from_source(collection, source_name= "pardubickenovinky.cz" , time_granularity="day"):
    date_format="%Y-%m-%d"
    if time_granularity == "month":
        date_format="%Y-%m"
    if time_granularity == "year":
        date_format="%Y"
        
    result = collection.aggregate([
        { "$match" : { "article.SourceCommonName" : source_name } },
        { "$unwind": { "path": "$article.Themes", "preserveNullAndEmptyArrays": False } },
        { "$unwind": { "path": "$article.Persons", "preserveNullAndEmptyArrays": False } },
        { "$group" : { "_id" : {"date":{ "$dateToString": { "format": date_format, "date": {"$dateFromString":{
            "dateString": {"$toString": {"$toLong" : "$article.DATE" }}, "format": "%Y%m%d%H%M%S"}}}},
                                "theme":"$article.Themes",
                                "person":"$article.Persons",
                                "lieu": "$event.ActionGeo_CountryCode"
                               }, 
                      "unique_article": {"$addToSet": "$article.GKGRECORDID"},
                      "V2Tone_list": { "$push" : "$article.V2Tone"}
                     }
        }, 
        { "$project" : {"_id" : 0,
                        "date": "$_id.date",
                        "theme":"$_id.theme",
                        "person":"$_id.person",
                        "lieu": "$_id.lieu",
                        "nb_distinct_article" : {"$size":"$unique_article"},
                        "mean_V2Tone" : { "$avg" : "$V2Tone_list"}
                       }
        }
    ])
    return pd.DataFrame(list(result))
       
df_article_from_source = article_from_source(mention_collection, source_name="lefigaro.fr" , time_granularity="month")
df_article_from_source

**QUESTION 4 :** Etudiez l’évolution des relations entre deux pays (specifiés en paramètre) au cours de l’année. Vous pouvez vous baser sur la langue de l’article, le ton moyen des articles, les themes plus souvent citées, les personnalités ou tout élément qui vous semble pertinent.

In [ ]:
mention_collection.create_index([("event.Actor1Geo_CountryCode", 1),
                                 ("event.Actor2Geo_CountryCode", 1) ], unique=False )

In [ ]:
%%time

def evolution_relation(collection, pays_A="RS", pays_B="UP"):
    result = collection.aggregate([
        { "$match": {"$or" : [{ 'event.Actor1Geo_CountryCode': pays_A, 'event.Actor2Geo_CountryCode':pays_B },
                              { 'event.Actor1Geo_CountryCode': pays_B, 'event.Actor2Geo_CountryCode':pays_A }
                             ]
                    }
        },
        { "$group" : { "_id" : {"actor1": "$event.Actor1Geo_CountryCode",
                                "date": { "$dateToString": { "format": "%Y-%m-%d", "date": {"$dateFromString":{
                                    "dateString": {"$toString": {"$toLong" : "$article.DATE" }}, "format": "%Y%m%d%H%M%S"}}}}
                               },
                      "V2Tone_list": { "$push" : "$article.V2Tone"}
                     }
        },
        { "$project" : {"actor1":"$_id.actor1", 
                        "date": "$_id.date",
                        "mean_V2Tone": {"$avg" : "$V2Tone_list"},
                        "_id" : 0 
                       }
        },
        {"$sort": {"date": 1}}
    ])
    return pd.DataFrame(list(result))

df_evolution_relation = evolution_relation(mention_collection, pays_A="UP", pays_B="RS")
df_evolution_relation

In [ ]:
fig = px.line(df_evolution_relation, x="date", y="mean_V2Tone", color="actor1", hover_name="actor1")
fig.show()